In [12]:
import pandas as pd
import numpy as np

This file was originally for frequency distributions for Latin names!
but we can also concat the other dfs / do model training after in the same file

In [13]:
df_indo = pd.read_csv('df_indo.csv')
df_malay = pd.read_csv('df_malay.csv')
df_viet = pd.read_csv('viet_df.csv')
df_cnrom = pd.read_csv('cnrom_df.csv')
df_cnchar = pd.read_csv('cnchar_df.csv')
# also import other dfs

### 1. Cleaning up column names

column names to KEEP: (10 so far)

* name_length
* avg_token_length
* num_tokens
* period_freq
* dash_freq
* apostrophe_freq
* space_freq
* unigrams_cosine_sim
* bigrams_cosine_sim
* language

In [14]:
# # finding what columns r in viet that aren't in indo
# print(df_indo.columns)
# print(df_viet.columns)
# [col for col in df_viet.columns if col not in df_indo.columns]

In [15]:
# idk if i can generalize this lol
# def rename_columns(df_ref, df_to_change):
#     diff_cols = [col for col in df_to_change.columns if col not in df_ref.columns]
#     new_names = 
#     df_to_change.rename(columns = {oldName : newName})

In [16]:
#This is where you rename columns to all match
df_viet.rename(columns = {'word_length': 'name_length'}, inplace = True)
df_cnrom.rename(columns = {'word_length': 'name_length'}, inplace = True)
df_cnchar.rename(columns = {'word_length': 'name_length'}, inplace = True)

### 4. Keeping numerical columns only for each dataset

In [17]:
# # DO FOR EACH DATASET
# # dropping non-numerical columns - this is why it would be good to have more numerical features since we don't have a lot
# df_indo = df_indo.select_dtypes(exclude = 'object')
# # df_indo

In [18]:
# # DROP FOR WHOLE DATASET
# # dropping trigrams for indo and malay
# df_indo.drop('trigrams_cosine_sim', inplace = True, axis = 1)
# df_malay.drop('trigrams_cosine_sim', inplace = True, axis = 1)

In [19]:
# # clean up viet / other dfs before combining
# df_viet.drop('trigrams', inplace = True, axis = 1)

### 5. Adding the language (label) to each dataset

In [20]:
df_indo['language'] = 'Indonesian'
df_malay['language'] = 'Malay'
df_viet['language'] = 'Vietnamese'
df_cnrom['language'] = 'Chinese (Romanized)'
df_cnchar['language'] = 'Chinese (Characters)'
# df_indo.head()

### 6. Combining all names to make one big dataset

In [21]:
# as you can see from output, we need the columns in the concatenated df (in this case, viet) to match
# it's okay if some values are NaN bc we'll drop all non-numerical columns anyway
merged_df = pd.concat([df_indo, df_malay, df_viet, df_cnrom, df_cnchar], ignore_index = True, join = 'outer')
merged_df

,fullname,original_fullname,alphabet,unigrams,bigrams,trigrams,char_ngrams,word_ngrams,name_length,avg_token_length,...,dash_freq,apostrophe_freq,space_freq,indiv_unigrams_fdist,indiv_bigrams_fdist,indiv_trigrams_fdist,unigrams_cosine_sim,bigrams_cosine_sim,trigrams_cosine_sim,language
0,supriyadi,Supriyadi,"['LATIN', 'LATIN', 'LATIN', 'LATIN', 'LATIN', ...","['s', 'u', 'p', 'r', 'i', 'y', 'a', 'd', 'i']","[('s', 'u'), ('u', 'p'), ('p', 'r'), ('r', 'i'...","[('s', 'u', 'p'), ('u', 'p', 'r'), ('p', 'r', ...","['s', 'u', 'p', 'r', 'i', 'y', 'a', 'd', 'i', ...",['supriyadi'],9,9.000000,...,0,0,0,[[0. 0. 0. 0. ...,[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]],0.664809,0.250640,0.085949,Indonesian
1,triyaningsih,Triyaningsih,"['LATIN', 'LATIN', 'LATIN', 'LATIN', 'LATIN', ...","['t', 'r', 'i', 'y', 'a', 'n', 'i', 'n', 'g', ...","[('t', 'r'), ('r', 'i'), ('i', 'y'), ('y', 'a'...","[('t', 'r', 'i'), ('r', 'i', 'y'), ('i', 'y', ...","['t', 'r', 'i', 'y', 'a', 'n', 'i', 'n', 'g', ...",['triyaningsih'],12,12.000000,...,0,0,0,[[0. 0. 0. 0. ...,[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]],0.686625,0.353292,0.117226,Indonesian
2,soerjadi,Soerjadi,"['LATIN', 'LATIN', 'LATIN', 'LATIN', 'LATIN', ...","['s', 'o', 'e', 'r', 'j', 'a', 'd', 'i']","[('s', 'o'), ('o', 'e'), ('e', 'r'), ('r', 'j'...","[('s', 'o', 'e'), ('o', 'e', 'r'), ('e', 'r', ...","['s', 'o', 'e', 'r', 'j', 'a', 'd', 'i', ('s',...",['soerjadi'],8,8.000000,...,0,0,0,[[0. 0. 0. 0. 0. 0.125 0. 0....,[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]],0.688312,0.197139,0.090295,Indonesian
3,undunsyah,Undunsyah,"['LATIN', 'LATIN', 'LATIN', 'LATIN', 'LATIN', ...","['u', 'n', 'd', 'u', 'n', 's', 'y', 'a', 'h']","[('u', 'n'), ('n', 'd'), ('d', 'u'), ('u', 'n'...","[('u', 'n', 'd'), ('n', 'd', 'u'), ('d', 'u', ...","['u', 'n', 'd', 'u', 'n', 's', 'y', 'a', 'h', ...",['undunsyah'],9,9.000000,...,0,0,0,[[0. 0. 0. 0. ...,[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]],0.581396,0.155386,0.060083,Indonesian
4,soeripto,Soeripto,"['LATIN', 'LATIN', 'LATIN', 'LATIN', 'LATIN', ...","['s', 'o', 'e', 'r', 'i', 'p', 't', 'o']","[('s', 'o'), ('o', 'e'), ('e', 'r'), ('r', 'i'...","[('s', 'o', 'e'), ('o', 'e', 'r'), ('e', 'r', ...","['s', 'o', 'e', 'r', 'i', 'p', 't', 'o', ('s',...",['soeripto'],8,8.000000,...,0,0,0,[[0. 0. 0. 0. 0. 0. 0. 0....,[[0. 0. 0. ... 0. 0. 0.]],[[0. 0. 0. ... 0. 0. 0.]],0.463215,0.176917,0.052811,Indonesian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37972,NaN,龚翔宇,"['CJK', 'CJK', 'CJK']","['g', 'o', 'n', 'g', ' ', 'x', 'i', 'a', 'n', ...","[('g', 'o'), ('o', 'n'), ('n', 'g'), ('g', ' '...","[('g', 'o', 'n'), ('o', 'n', 'g'), ('n', 'g', ...","[('g',), ('o',), ('n',), ('g',), (' ',), ('x',...",NaN,13,3.666667,...,0,0,2,[[0.15384615 0.07692308 0. 0. ...,[[0. 0. 0. 0. ...,NaN,0.844943,0.665759,NaN,Chinese (Characters)
37973,NaN,龚育之,"['CJK', 'CJK', 'CJK']","['g', 'o', 'n', 'g', ' ', 'y', 'u', ' ', 'z', ...","[('g', 'o'), ('o', 'n'), ('n', 'g'), ('g', ' '...","[('g', 'o', 'n'), ('o', 'n', 'g'), ('n', 'g', ...","[('g',), ('o',), ('n',), ('g',), (' ',), ('y',...",NaN,11,3.000000,...,0,0,2,[[0.18181818 0. 0. 0. ...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,NaN,0.837370,0.501902,NaN,Chinese (Characters)
37974,NaN,龚蓓苾,"['CJK', 'CJK', 'CJK']","['g', 'o', 'n', 'g', ' ', 'b', 'e', 'i', ' ', ...","[('g', 'o'), ('o', 'n'), ('n', 'g'), ('g', ' '...","[('g', 'o', 'n'), ('o', 'n', 'g'), ('n', 'g', ...","[('g',), ('o',), ('n',), ('g',), (' ',), ('b',...",NaN,11,3.000000,...,0,0,2,[[0.18181818 0. 0.18181818 0. ...,[[0. 0. 0.2 0. 0. 0. 0. 0. 0. 0. 0. ...,NaN,0.729526,0.363754,NaN,Chinese (Characters)
37975,NaN,龚贤永,"['CJK', 'CJK', 'CJK']","['g', 'o', 'n', 'g', ' ', 'x', 'i', 'a', 'n', ...","[('g', 'o'), ('o', 'n'), ('n', 'g'), ('g', ' '...","[('g', 'o', 'n'), ('o', 'n', 'g'), ('n', 'g', ...","[('g',), ('o',), ('n',), ('g',), (' ',), ('x',...",NaN,14,4.000000,...,0,0,2,[[0.14285714 0.07142857 0. 0. ...,[[0. 0. 0. 0. ...,NaN,0.808012,0.637316,Na

### 7. Keeping numerical columns only for each dataset (Same process as step 4, except we don't have to repeat lines of code...)

In [22]:
merged_df = merged_df.select_dtypes(exclude = 'object')
merged_df.drop('trigrams_cosine_sim', inplace = True, axis = 1)
merged_df

,name_length,avg_token_length,num_tokens,period_freq,dash_freq,apostrophe_freq,space_freq,unigrams_cosine_sim,bigrams_cosine_sim
0,9,9.000000,1,0,0,0,0,0.664809,0.250640
1,12,12.000000,1,0,0,0,0,0.686625,0.353292
2,8,8.000000,1,0,0,0,0,0.688312,0.197139
3,9,9.000000,1,0,0,0,0,0.581396,0.155386
4,8,8.000000,1,0,0,0,0,0.463215,0.176917
...,...,...,...,...,...,...,...,...,...
37972,13,3.666667,3,0,0,0,2,0.844943,0.665759
37973,11,3.000000,3,0,0,0,2,0.837370,0.501902
37974,11,3.000000,3,0,0,0,2,0.729526,0.363754
37975,14,4.000000,3,0,0,0,2,0.808012,0.637316


### Steps

Next steps:
1. Clean up columns so we can combine dataframes into one (focus on making an all-Latin dataset first)
    - do not combine in this step
2. Frequency distributions for Latin names -> redo
3. Add a_hat_freq
4. Only keep numerical columns
    - turn some categorical features -> numerical so we have more things to feed into model
5. Add in label (language) for each dataset
6. Combine Latin and non-Latin names to make one big dataset
    - may need to repeat some of the above steps for non-Latin names
7. Train test split
8. MODEL TRAINING!
9. Model evaluation

Reminder:
- We decided to keep period_freq, dash_freq, apostrophe_freq for now. After our first run of model training, we can remove them to see if it improves the performance

**You can work on these steps out of order** (act as if the previous steps r there), but in the end we ideally want all of these steps implemented in this order.

For example, you could write the code for model training and train the model on one or a few datasets. Later on, we'll just replace the variables you used with the ones containing all the languages/names.